"""Имеются данные adult.csv (см. в материалах к занятию).
Целевой переменной является уровень дохода income (крайний правый столбец).
Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели."""

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [2]:
adult = pd.read_csv( 'adult.csv' )
adult.head(10)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [3]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
selectedColumns = adult[ [ 'age', 'race', 'gender', 'hours-per-week', 'education', 'educational-num', 'relationship']]

In [5]:
X = pd.get_dummies(selectedColumns, columns = ['race', 'gender', 'education', 'educational-num', 'relationship'])

In [6]:
X.head()

,age,hours-per-week,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,gender_Female,gender_Male,education_10th,...,educational-num_13,educational-num_14,educational-num_15,educational-num_16,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife
0,25,40,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,38,50,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,28,40,0,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,44,40,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,18,30,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
le = LabelEncoder()

In [8]:
le.fit( adult['income'] )

LabelEncoder()

In [9]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [10]:
y = pd.Series( le.transform( adult['income'] ) )
y

0        0
1        0
2        1
3        1
4        0
        ..
48837    0
48838    1
48839    0
48840    0
48841    1
Length: 48842, dtype: int32

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [12]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
print(y_pred_test)

[0 0 1 ... 0 0 0]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
model.score(X_test, y_test)

0.8238473507493244

In [14]:
model_SVC = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [15]:
model_SVC.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [16]:
model_SVC.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [17]:
model_SVC.score(X_test, y_test)

0.8229465236262387